In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from tqdm import tqdm

In [2]:
# -------------------------------------------------------------
# Define Device
# -------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------------------------------
# Load Pretrained ResNet50 for Feature Extraction
# -------------------------------------------------------------
resnet_model = resnet50(weights=True)
resnet_model = torch.nn.Sequential(*list(resnet_model.children())[:-1])  # Remove classification layer
resnet_model.eval().to(device)

# -------------------------------------------------------------
# Define Image Transformation
# -------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

C:\Users\PAWANESH\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# -------------------------------------------------------------
# Feature Extraction Function
# -------------------------------------------------------------
def extract_features(image_path, transform, device=device):
    """Extract ResNet50 features from a single image."""
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)  # Convert to tensor and send to GPU

        # Extract features using ResNet50
        with torch.no_grad():
            features = resnet_model(image).squeeze().cpu().numpy()
        
        return features
    except Exception as e:
        print(f"Skipping frame: {image_path} due to error: {e}")
        return None



# -------------------------------------------------------------
# Paths to RGB and Label CSV
# -------------------------------------------------------------
rgb_path = "D:/Datasets/Datasets/GTEA/GTEA_NEW/S2_Cheese_C1"
label_csv_path = "D:/Datasets/Datasets/GTEA/GTEA_NEW/Label_CSV/S2_Cheese_C1.csv"
output_csv = "Feature_ResNet50Only/Feature_S2_Cheese_C1_GTEA.csv"
# Load label CSV
labels_df = pd.read_csv(label_csv_path)

In [13]:
# -------------------------------------------------------------
# Extract Features and Save to CSV
# -------------------------------------------------------------
S = 1  # Sampling every 100th frame
features_list = []

# Get all RGB frames (sampled every 100th frame)
all_frames = sorted(os.listdir(rgb_path))[::S]

for frame in tqdm(all_frames, desc="Extracting RGB Features"):
    rgb_frame_path = os.path.join(rgb_path, frame)
    
    # Extract features for RGB frame
    rgb_features = extract_features(rgb_frame_path, transform, device)
    
    if rgb_features is not None:
        # Extract frame number from the frame name
        frame_number = int(frame.split('_')[-1].split('.')[0])
        
        # Get corresponding label information
        label_row = labels_df[(labels_df['StartFrame'] <= frame_number) & (labels_df['EndFrame'] >= frame_number)]
        
        if not label_row.empty:
            action_label = label_row.iloc[0]['ActionLabel']
        else:
            action_label = 0  # Default if no label found
        
        # Append features and labels to the list
        features_list.append([frame, action_label] + rgb_features.tolist())

# -------------------------------------------------------------
# Save Features to CSV
# -------------------------------------------------------------
if len(features_list) == 0:
    raise ValueError("No valid features extracted. Please check the dataset paths and feature extraction function.")

# Define column names
columns = ["Frame", "ActionLabel"] + [f"Feature_{i}" for i in range(len(rgb_features))]
df = pd.DataFrame(features_list, columns=columns)
df.to_csv(output_csv, index=False)

print(f"Feature extraction completed successfully! Saved to {output_csv}")

Extracting RGB Features: 100%|███████████████████████████████████████████████████████| 634/634 [00:12<00:00, 51.59it/s]


Feature extraction completed successfully! Saved to Feature_ResNet50Only/Feature_S2_Cheese_C1_GTEA.csv


In [10]:
import pandas as pd
df=pd.read_csv("Feature_ResNet50Only/Feature_P01_04_EpicKitchen.csv")
df

,Frame,ActionLabel,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,...,Feature_2038,Feature_2039,Feature_2040,Feature_2041,Feature_2042,Feature_2043,Feature_2044,Feature_2045,Feature_2046,Feature_2047
0,frame_00000.jpg,0,0.760674,0.420604,0.121561,0.470687,2.164679,0.641249,0.017902,2.015719,...,1.784048,0.538451,0.886709,0.858122,0.282893,0.011314,0.025618,0.099124,0.913920,0.598013
1,frame_00010.jpg,0,0.463165,0.644922,0.376398,0.380077,2.298037,0.658172,0.073457,1.484530,...,1.415602,0.575273,0.306665,0.593542,0.336238,0.014430,0.069869,0.088369,0.972922,0.364018
2,frame_00020.jpg,0,0.274978,0.523290,0.261900,0.402526,2.024251,0.757041,0.043806,2.368922,...,1.557384,0.631702,0.414883,0.678494,0.361808,0.016541,0.049876,0.020578,0.734077,0.433087
3,frame_00030.jpg,0,0.421221,0.505983,0.517255,0.346130,1.705878,0.624451,0.148247,1.973320,...,0.981156,0.312534,0.358885,0.701922,0.375918,0.003701,0.101886,0.041058,0.754904,0.582021
4,frame_00040.jpg,0,0.429257,0.542538,0.401359,0.296634,1.970009,0.558138,0.248166,0.968274,...,1.158898,0.162853,0.299160,0.814168,0.289126,0.013347,0.080683,0.053804,1.033205,0.362499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,frame_06260.jpg,0,0.052075,0.510224,0.514524,0.189549,0.908797,0.510574,0.653865,0.171677,...,0.522134,0.389426,0.663176,0.374699,0.058822,0.100730,0.029890,0.099107,0.003079,0.085428
627,frame_06270.jpg,0,0.029805,0.126756,0.335512,0.133975,1.172383,0.656238,0.681204,0.354457,...,0.358023,0.274439,0.542280,0.209008,0.111753,0.106518,0.014506,0.116043,0.100046,0.157220
628,frame_06280.jpg,0,0.015115,0.530739,0.043024,0.242603,0.989694,0.963369,0.163384,0.122910,...,0.227367,0.800139,0.197479,0.093685,0.168871,0.100993,0.000000,0.160916,0.113592,0.066160
629,frame_06290.jpg,0,0.072961,0.432900,0.298266,0.356081,1.086133,0.954952,0.118512,0.070932,...,0.249096,0.435129,0.258884,0.193486,0.085338,0.113787,0.111505,0.086939,0.042602,0.062170
